In [1]:
import pandas as pd
import talib as TA
import pandas_ta as ta
import matplotlib.pyplot as plt
from backtesting import Strategy, Backtest

In [2]:
'''
This strategy takes into account RSI for entry signals and a exponential moving average for trend detection.
is intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.
The backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.
'''

'\nThis strategy takes into account RSI for entry signals and a exponential moving average for trend detection.\nis intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.\nThe backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.\n'

READ & CLEAN THE DATA SET

In [4]:
df = pd.read_csv('/mnt/c/Users/samue/Documents/ALGORITHMIC-TRADING-PROJECTS/backtesting strategies/data_sets/AUDUSD_Candlestick_15_M_BID_21.04.2004-16.04.2022.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Open,High,Low,Close,Volume
0,2004-04-21 00:00:00,0.73057,0.73215,0.73007,0.73163,36625.7781
1,2004-04-21 00:15:00,0.73118,0.73138,0.72993,0.73033,24471.4404
2,2004-04-21 00:30:00,0.73034,0.73155,0.72995,0.73072,37426.0350
3,2004-04-21 00:45:00,0.73087,0.73113,0.73016,0.73031,35051.6650
4,2004-04-21 01:00:00,0.73076,0.73105,0.72924,0.73030,28241.7052
...,...,...,...,...,...,...
445622,2022-04-15 14:45:00,0.73921,0.73925,0.73905,0.73922,254.3800
445623,2022-04-15 15:00:00,0.73922,0.73929,0.73902,0.73921,288.3600
445624,2022-04-15 15:15:00,0.73922,0.73949,0.73916,0.73921,153.3400
445625,2022-04-15 15:30:00,0.73921,0.73926,0.73900,0.73915,602.3500


SLICING THE DATASET

In [5]:
#60% of the dataset
df60 = df[0:270000]  
#40% of the dataset
df40 = df[270000:449192]  
df60.dropna()
df40.dropna()

,Date,Open,High,Low,Close,Volume
270000,2015-01-04 03:45:00,0.75951,0.75954,0.75823,0.75892,1996.60
270001,2015-01-04 04:00:00,0.75890,0.75909,0.75833,0.75890,1818.58
270002,2015-01-04 04:15:00,0.75889,0.75932,0.75884,0.75890,1338.59
270003,2015-01-04 04:30:00,0.75891,0.75958,0.75891,0.75938,1427.35
270004,2015-01-04 04:45:00,0.75940,0.75972,0.75932,0.75963,855.34
...,...,...,...,...,...,...
445622,2022-04-15 14:45:00,0.73921,0.73925,0.73905,0.73922,254.38
445623,2022-04-15 15:00:00,0.73922,0.73929,0.73902,0.73921,288.36
445624,2022-04-15 15:15:00,0.73922,0.73949,0.73916,0.73921,153.34
445625,2022-04-15 15:30:00,0.73921,0.73926,0.73900,0.73915,602.35


BACKTEST USING ATR RELATED SL&TP

In [28]:
class MyStrat(Strategy):
    ema_window = 200
    rsi_window = 2
    atr_window = 14
    riskpct = 0.2
    
    def init(self):
        super().init()
        self.ema = self.I(TA.EMA, self.data.Close, self.ema_window)
        self.rsi = self.I(TA.RSI, self.data.Close, self.rsi_window)   
        self.atr = self.I(TA.ATR, self.data.Close, self.data.High, self.data.Low, self.atr_window) 

    def next(self):
        super().next()
        slatr = 1.2*self.atr[-1]
        TPSLRatio = 2
        price = self.data.Close[-1] 
        
        if self.position:
            self.position.close()
            
        if self.data.Close >= self.ema and self.rsi <= 10:
            sl1 = price - slatr
            tp1 = price + slatr*TPSLRatio    # 1e-4 is 1 pip
             
            
            self.buy(sl=sl1, tp=tp1, size= self.riskpct)
            
        if self.data.Close <= self.ema and self.rsi >= 90:
            sl2 = price + slatr
            tp2 = price - slatr*TPSLRatio    # 1e-4 is 1 pip
            
                                    
            self.sell(sl=sl2, tp=tp2, size= self.riskpct)

bt = Backtest(df40, MyStrat, cash=100, margin=1/50, commission = 0.000, trade_on_close=False)
stat = bt.run()
print(stat)
trades_size = stat['_trades']#['Size'] 
print(trades_size) # shows the size of the trades
# #plt.bar(trades_size, height=trades_size.count())
# #plot = bt.plot(plot_volume=False)
# #print(plot)

/tmp/ipykernel_510/2879234067.py:36: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df40, MyStrat, cash=100, margin=1/50, commission = 0.000, trade_on_close=False)


Start                                270000.0
End                                  445626.0
Duration                             175626.0
Exposure Time [%]                   12.930814
Equity Final [$]                 15040.246235
Equity Peak [$]                   16604.85324
Return [%]                       14940.246235
Buy & Hold Return [%]               -2.627418
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.217352
Avg. Drawdown [%]                   -1.172666
Max. Drawdown Duration                11120.0
Avg. Drawdown Duration             194.783518
# Trades                              14668.0
Win Rate [%]                        53.538315
Best Trade [%]                       1.619609
Worst Trade [%]                     -0.930914
Avg. Trade [%]                    

BACKTEST USING SIZE CALCULATION BY RISK% AND SL SIZE

In [24]:
# class MyStrat(Strategy):
#     ema_window = 200
#     rsi_window = 2
#     riskpct = 0.02
#     contract_size = 100000
    
#     def init(self):
#         super().init()
#         self.ema = self.I(TA.EMA, self.data.Close, self.ema_window)
#         self.rsi = self.I(TA.RSI, self.data.Close, self.rsi_window)   

#     def next(self):
#         super().next()
        
#         if self.position:
#             self.position.close()
            
#         if self.data.Close >= self.ema and self.rsi <= 10:
#             o, h, l, c = self.data.Open[-1], self.data.High[0], self.data.Low[0], self.data.Close[-1]
#             size1 = (self.equity*self.riskpct)/((o - l)*self.contract_size)     
            
#             self.buy(sl = l - (5e-4), tp = h + (100e-4), size=(size1))
            
#         if self.data.Close <= self.ema and self.rsi >= 90:
#             o, h, l, c = self.data.Open[-1], self.data.High[0], self.data.Low[0], self.data.Close[-1]
#             size2 = (self.equity*self.riskpct)/((h - c)*self.contract_size)
                                    
#             self.sell(sl = h + (5e-4), tp = l - (100e-4), size = (size2))

# bt = Backtest(df40, MyStrat, cash=100, margin=1/50, commission = 0.00, trade_on_close=False)
# stat = bt.run()
# print(stat)
# trades_size = stat['_trades']#['Size'] 
# print(trades_size) # shows the size of the trades
# # plt.bar(trades_size, height=trades_size.count())
# # plot = bt.plot(plot_volume=False)
# # print(plot)

/tmp/ipykernel_510/2157516042.py:30: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df40, MyStrat, cash=100, margin=1/50, commission = 0.00, trade_on_close=False)


AssertionError: size must be a positive fraction of equity, or a positive whole number of units

OPTIMIZATION

In [ ]:
# Since Optimization takes some time to analize the data, I already have done it and the best parameters are: EMA=200, RSI=2

# stats = bt.optimize(
#     ema_window = [50, 200],
#     rsi_window = range(2, 4),
#     maximize = 'Equity Final [$]',
# )

# print(stats)
# print(stats._strategy)
# bt.plot(plot_volume=False, plot_pl=False)

CALCULATION OF POSITION SIZE BY RISK% AND SL SIZE

In [ ]:
# c, h, o, l = self.data.Close[-1], self.data.High[-1], self.data.Open[-1], self.data.Low[-1]
# riskpct = 0.002
# contract_size = 100000
# size2 = (self.equity*riskpct)//(h - c)*contract_size
# size1 = (self.equity*riskpct)//(o - l)*contract_size

CALCULATION OF SL ON LOW AND HIGH OF SIGNAL CANDLE 

In [ ]:
# c, h, o, l = self.data.Close[-1], self.data.High[-1], self.data.Open[-1], self.data.Low[-1]
# self.sell(limit=c, sl=h+4e-4, tp=c - 2*(c-1), size=riskpct)
# self.buy(limit=c, sl=l-4e-4, tp=c + 2*(c-1), size=riskpct)